In [130]:
import torch
import pandas as pd
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [131]:
torch.cuda.is_available()

True

In [132]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [133]:
train = pd.read_csv('/home/jwjung/Recsys/ml-100k/ml-100k/ua.base'
,sep = '\t', names = ['user','movie','rating','ts'])
test = pd.read_csv('/home/jwjung/Recsys/ml-100k/ml-100k/ua.test'
,sep = '\t', names = ['user','movie','rating','ts'])

train.head()

,user,movie,rating,ts
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [134]:
items = torch.LongTensor(train['movie'])
users = torch.LongTensor(train['user'])
ratings = torch.FloatTensor(train['rating'])

In [135]:
# items = items.to(device)
# users = users.to(device)
# ratings = ratings.to(device)

In [136]:
items_test = torch.LongTensor(test['movie'])
users_test = torch.LongTensor(test['user'])
ratings_test = torch.FloatTensor(test['rating'])

In [137]:
# items_test = items_test.to(device)
# users_test = users_test.to(device)
# ratings_test = ratings_test.to(device)

In [138]:
latent_dim = 100

num_items = items.max() + 1
num_users = users.max() + 1

torch.manual_seed(23)
P = torch.randn(num_items, latent_dim, requires_grad = True)
Q = torch.randn(num_users, latent_dim, requires_grad = True)

optimizer = torch.optim.Adam([P,Q], lr = 0.1)

# P = P.to(device)
# Q = Q.to(device)

for epoch in range(1000):
    """
    1. P[items]는 P에서 해당하는 item의 벡터만 따로 뽑아오는 의미
    P가 하나의 look-up table로서 작용
    2. torch.sum에서 dim 없으면 전체 원소 합 도출
    -> dim = 1은 2차원 기준으로 row 단위로 합 구하라는 의미
    -> 여기서 dim = 1로 sum 하는 이유: 각 벡터의 feature 별 elementwise product 값이 도출된 결과가
    rating matrix의 하나의 element로 존재 -> 그 값을 전부 더해줘야 하나의 rating 도출 가능
    ex) feature dim = 3일 때 P*Q 결과: [-1,2,2] => 각 feature 별 점수 합친 3이 pred_rating 임!!
    3. P * Q 는 elementwise product -> dot product 아님!!! (dot product는 torch.matmul 사용)
    """
    hypothesis = torch.sum(P[items] * Q[users],dim = 1)
    cost = F.mse_loss(hypothesis, ratings)

    optimizer.zero_grad() # zero_grad 처리 안해줄 시, 이전 epoch에서 구한 grad값이 남아있어서 값이 더해짐! -> 그 값날린다 생각 (초기화)
    cost.backward() # backpropagation 진행
    optimizer.step() # backprop 진행 시 나온 변화도로 parameter 조정

    if epoch % 100 == 0:
        print(f"epoch: {epoch}, cost: {cost.item()}")

epoch: 0, cost: 113.9721450805664
epoch: 100, cost: 0.07595525681972504
epoch: 200, cost: 0.026424439623951912
epoch: 300, cost: 0.012290881015360355
epoch: 400, cost: 0.006570287514477968
epoch: 500, cost: 0.0038495203480124474
epoch: 600, cost: 0.002429467160254717
epoch: 700, cost: 0.0015924581093713641
epoch: 800, cost: 0.001120501197874546
epoch: 900, cost: 0.0007672904175706208
